In [1]:
from transformers import LlamaTokenizer, LlamaForSequenceClassification, LlamaConfig

2024-01-06 14:37:39.254268: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-06 14:37:39.254341: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-06 14:37:39.254377: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-06 14:37:39.264500: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import warnings
import logging
import os
warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['WANDB_PROJECT'] = 'kaggle-llm-2023-llama-2-7b-test-train'


In [3]:
config_dir = '/home/peng_sun2/s3shared/kaggle/llm-2023/Llama-2-7b-chat-hf/config'
tokenizer_dir = '/home/peng_sun2/s3shared/kaggle/llm-2023/Llama-2-7b-chat-hf/tokenizer'
model_dir = '/home/peng_sun2/s3shared/kaggle/llm-2023/Llama-2-7b-chat-hf/model'
config= LlamaConfig.from_pretrained(config_dir)
tokenizer = LlamaTokenizer.from_pretrained(tokenizer_dir)

## set up the pad token
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "left"
config.pad_token_id = tokenizer.pad_token_id

## set up the number of labels
config.num_labels = 2

In [4]:
model = LlamaForSequenceClassification.from_pretrained(model_dir, config = config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
type(tokenizer)

transformers.models.llama.tokenization_llama.LlamaTokenizer

In [6]:
def tokenize(text, tokenizer, config):
    sep = tokenizer.sep_token
    tokenized = tokenizer(
        text,
        padding = True,
        truncation = True,
        max_length = 1600
    )
    return {
        **tokenized
    }

In [7]:
text_sentence = "This is a test sentence"
text_sent_tok = tokenize(text_sentence, tokenizer, config)

In [8]:
import torch
input_tensor = {
    'input_ids':torch.tensor(text_sent_tok['input_ids'], dtype = torch.long).view(1, -1),
    'attention_mask': torch.tensor(text_sent_tok['attention_mask'], dtype= torch.long).view(1, -1)
}

In [9]:
preds = model(input_ids = input_tensor['input_ids'], 
      attention_mask = input_tensor['attention_mask'])

In [10]:
preds[0]

tensor([[ 5.3142, -4.3097]], grad_fn=<IndexBackward0>)

In [11]:
from transformers import HfArgumentParser
from transformers import TrainingArguments

## Read in the training data

In [12]:
import pandas as pd

In [13]:

test = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/test_essays.csv')
sub = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/sample_submission.csv')
org_train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/train_essays.csv')

train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/daigt/train_v2_drcat_02.csv', sep=',')
display(train.shape);

train_gen_data = pd.read_parquet('/home/peng_sun2/s3shared/kaggle/llm-2023/external_data/gen_data_21122023.parquet')
display(train_gen_data.shape)

train = train.drop_duplicates(subset = ['text'])
train.reset_index(drop = True, inplace = True)

### append the generated data
# train = pd.concat([train, train_gen_data], axis = 0).reset_index(drop = True)
# display(train.shape)

(44868, 5)

(18894, 5)

## Split the training data using groupkfold

In [14]:
train.head(3)

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False


In [15]:
from sklearn.model_selection import GroupKFold

group_kfold = GroupKFold(n_splits = 5)
groups = train.prompt_name
group_kfold.get_n_splits(train['text'], train['label'], groups)

5

In [16]:
def tokenize_train(df, tokenizer):
    tokenized = tokenizer(
        df['text'],
        padding = True,
        truncation = True,
        max_length = 1600
    )
    
    labels = df['label']
    return {
        **tokenized,
        'labels': labels
    }

In [17]:
for i, (train_idx, test_idx) in enumerate(group_kfold.split(train['text'], train['label'], groups)):
    print(f'Fold {i}: ')
    print(f'Train index shape: {train_idx.shape}, group = {groups[train_idx].unique()}')
    print(f'Test index shape: {test_idx.shape}, group = {groups[test_idx].unique()}')

Fold 0: 
Train index shape: (35612,), group = ['Car-free cities' 'Summer projects' '"A Cowboy Who Rode the Waves"'
 'Mandatory extracurricular activities' 'Exploring Venus'
 'Facial action coding system' 'The Face on Mars' 'Community service'
 'Grades for extracurricular activities' 'Driverless cars'
 'Does the electoral college work?' 'Seeking multiple opinions']
Test index shape: (9256,), group = ['Phones and driving' 'Cell phones at school' 'Distance learning']
Fold 1: 
Train index shape: (35623,), group = ['Phones and driving' 'Car-free cities' 'Summer projects'
 '"A Cowboy Who Rode the Waves"' 'Mandatory extracurricular activities'
 'Facial action coding system' 'Community service'
 'Grades for extracurricular activities' 'Driverless cars'
 'Does the electoral college work?' 'Cell phones at school'
 'Distance learning']
Test index shape: (9245,), group = ['Exploring Venus' 'The Face on Mars' 'Seeking multiple opinions']
Fold 2: 
Train index shape: (35809,), group = ['Phones and dr

In [18]:
from datasets import Dataset, disable_progress_bar

In [19]:
for i, (train_idx, test_idx) in enumerate(group_kfold.split(train['text'], train['label'], groups)):
    train_ds = Dataset.from_pandas(train.loc[train_idx, :])
    val_ds = Dataset.from_pandas(train.loc[test_idx, :])
    
    train_ds = train_ds.map(
        tokenize_train,
        batched = False,
        num_proc = 4,
        fn_kwargs = {'tokenizer': tokenizer}
    )
    
    val_ds = train_ds.map(
        tokenize_train,
        batched = False,
        num_proc = 4,
        fn_kwargs = {'tokenizer': tokenizer}
    )

Map (num_proc=4):   0%|          | 0/35612 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/35612 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/35623 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/35623 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/35809 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/35809 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/35837 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/35837 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36591 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36591 [00:00<?, ? examples/s]

In [20]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(
    tokenizer = tokenizer,
    pad_to_multiple_of=16 ,
)



In [21]:
from typing import Optional
from dataclasses import dataclass, field

@dataclass
class Config:
    add_prompt_text: Optional[bool] = field(
        default=False,
        metadata = {"help": "Add prompt text into input."}
    )
    
    max_seq_length: Optional[int] = field(
        default = 1600,
        metadata = {'help': 'max sequence length'}
    )
    fold: Optional[int] = field(
        default=0,
        metadata={"help": "Fold"},
    )

    num_proc: Optional[int] = field(
        default=4,
        metadata={"help": "Number of processes"},
    )

    dropout: Optional[float] = field(
        default=0.,
        metadata={"help": "Amount of dropout to apply"},
    )
    max_position_embeddings: Optional[int] = field(
        default=1600,
        #default=512,
        #default = 512,
        metadata={"help": "Amount of dropout to apply"},
    ) 
        
    data_dir: Optional[str] = field(
        default="./",
        metadata={"help": "Data directory"},
    )
        
    max_seq_length: Optional[int] = field(
        default=1600,
        #default = 512,
        #default = 512,
        metadata={"help": "Max sequence length"},
    )

    add_prompt_question: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt question into input"},
    )

    add_prompt_text: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt text into input"},
    )

    fold: Optional[int] = field(
        default=0,
        metadata={"help": "Fold"},
    )

    num_proc: Optional[int] = field(
        default=4,
        metadata={"help": "Number of processes"},
    )

    dropout: Optional[float] = field(
        default=0.,
        metadata={"help": "Amount of dropout to apply"},
    )
    max_position_embeddings: Optional[int] = field(
        default=1600,
        #default=512,
        #default = 512,
        metadata={"help": "Amount of dropout to apply"},
    )
        
#     output_dir: Optional[str] = field(
#         default= "./",
#         metadata={"help": "Output directory"}
#     )
        

In [22]:
config.update({
    'hidden_dropout_prob': 0,
    'attention_probs_dropout_prob': 0,
    'num_labels': 2,
    'problem_type': 'single_label_classification',
    'max_position_embeddings': 1600,
    'num_proc': 4
})

In [23]:
train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/daigt/train_v2_drcat_02.csv', sep=',')
display(train.shape);

train_gen_data = pd.read_parquet('/home/peng_sun2/s3shared/kaggle/llm-2023/external_data/gen_data_21122023.parquet')
display(train_gen_data.shape)

train = train.drop_duplicates(subset = ['text'])
train.reset_index(drop = True, inplace = True)
display(train.shape)

(44868, 5)

(18894, 5)

(44868, 5)

### Loss func

In [24]:
from sklearn import metrics
def compute_auc(eval_pred):
    preds, labels = eval_pred;
    fpr, tpr, thresholds = metrics.roc_curve(labels, preds, pos_label = 1)
    auc = metrics.auc(fpr, tpr)
    return {
        'auc': auc
    }

### Test train-val split

In [25]:
train_ds = Dataset.from_pandas(train.iloc[:30000, :])
val_ds = Dataset.from_pandas(train.iloc[30000:, :])

In [26]:
train_ds = train_ds.map(
    tokenize_train,
    batched = False,
    num_proc = config.num_proc,
    fn_kwargs = {'tokenizer': tokenizer}
)

val_ds = val_ds.map(
    tokenize_train,
    batched = False,
    num_proc = config.num_proc,
    fn_kwargs = {'tokenizer': tokenizer}
)

Map (num_proc=4):   0%|          | 0/30000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/14868 [00:00<?, ? examples/s]

In [27]:
data_collator = DataCollatorWithPadding(
        tokenizer=tokenize_train,
        pad_to_multiple_of=16
    )

In [28]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",  # Output directory for model checkpoints and predictions
    num_train_epochs=4,       # Number of training epochs
    per_device_train_batch_size=8,  # Batch size per GPU/TPU core for training
    per_device_eval_batch_size=8,   # Batch size per GPU/TPU core for evaluation
    learning_rate=6e-6,
    weight_decay=1e-4,        # Weight decay for regularization
    warmup_ratio=0,
    lr_scheduler_type='cosine',
    optim='adamw_torch',
    logging_dir='./logs',     # Directory for Tensorboard logs
    logging_strategy='steps',  # Log every `logging_steps`
    logging_steps=150,         # Log every 100 steps
    evaluation_strategy='steps',  # Evaluate every `eval_steps`
    eval_steps=150,             # Evaluate every 500 steps
    save_strategy='epoch',      # Save a checkpoint every epoch
    save_total_limit=1,
    save_steps=150,            # Save a checkpoint every 1000 steps
    report_to="wandb",          # Integration with Weights & Biases for logging
    run_name="test-llm-2023-llama2-run",     # Name for the current run
    metric_for_best_model='auc'
)

In [36]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_ds,
    eval_dataset = val_ds,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_auc
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 15.78 GiB total capacity; 14.72 GiB already allocated; 81.94 MiB free; 14.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF